#### Ссылка на корпус: https://drive.google.com/drive/u/0/folders/1BMbFqjGr-QsH5YX-mPd9P3653qf7MI-a

In [1]:
import requests
import urllib
from bs4 import BeautifulSoup
import random
import time
from tqdm import tqdm_notebook as tqdm
import re
import unicodedata
import collections

In [2]:
def bs(url):
    req = requests.get(url)
    html= req.text
    soup = BeautifulSoup(html, "lxml")
    return soup

In [3]:
def clean(text):
    text = re.sub('\n', '', text)
    text = unicodedata.normalize("NFKD", text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

In [4]:
def clean_info(text):
    text = re.sub('(\n)+', ' ', text)
    text = unicodedata.normalize("NFKD", text)
    text = re.sub('Адрес|Компания|Контактное лицо', ' ', text)
    text = re.sub('( ){2,}', ', ', text)
    text = text.strip()[2:-1]
    return text

In [5]:
def try_except(soup, _tag, _class, _group='class'):
    try:
        item = soup.find(_tag, attrs={_group:_class}).text
    except:
        item = 'NAN'
    return item

In [6]:
def find_info(soup, url):
    text = ''
    title = try_except(soup, 'span', 'title-info-title-text')
    num_date = try_except(soup, 'div', 'title-info-metadata-item')
    price = try_except(soup, 'span', 'price-value-string js-price-value-string')
    info_author = try_except(soup, 'div', 'item-view-seller-info')
    dog_params = try_except(soup, 'div', 'item-params')
    dog_location = try_except(soup, 'div', 'item-map-location')
    dog_description = try_except(soup, 'div', 'description', _group='itemprop')
    text = 'Название: ' + clean(title) + '\n' + \
    'Номер и дата объявления: ' + clean(num_date) + '\n' + \
    'Информация об авторе: ' + clean_info(info_author) + '\n' + \
    'Адрес собаки: ' + clean(dog_location).replace('Адрес: ', '').replace(' Посмотреть карту', '') + '\n' + \
    clean(dog_params) + '\n' + \
    'Цена: ' + clean(price) + '\n' + \
    'Описание собаки: ' + clean(dog_description) + '\n' + \
    'URL: ' + url
    return text

In [7]:
def write_file(text, name):
    name = name.replace('/', '$')#.replace(':', '$')
    file = open('/Users/irene/Downloads/IR/avito_dogs/' + name + ".txt", "w") 
    file.write(text)
    file.close()

#### Пример

In [8]:
url_of_article = '/sankt-peterburg/sobaki/shpits_1602362094'
begin = 'https://www.avito.ru'
clean_url = begin + url_of_article
soup_page = bs(clean_url)
text = find_info(soup_page, clean_url)
write_file(text, url_of_article)

In [9]:
text

'Название: Шпиц\nНомер и дата объявления: No 1602362094, размещено вчера в 12:50\nИнформация об авторе: Маленький Ангел, На Авито c августа 2011, Завершено 200 объявлений, Публичный профиль, 6 объявлений пользователя, Светлана, Санкт-Петербург, м. Парк Победы\nАдрес собаки: Санкт-Петербург, м. Парк Победы\nПорода: шпиц\nЦена: 25 000 ₽\nОписание собаки: Щенки померанского шпица 2мес., документы РКФ, привиты по возрасту. Папа 💯% Америка и мама живут в одном доме.Консультации заводчика по вопросам ухода и воспитания пожизненно. ДИАПАЗОН ЦЕН.\nURL: https://www.avito.ru/sankt-peterburg/sobaki/shpits_1602362094'

#### Выкачиваем корпус

In [ ]:
for x in tqdm(range(1, 101)):
    sec1 = random.uniform(0, 10)
    url = 'https://www.avito.ru/rossiya/sobaki?p=' + str(x) + '&view=list'
    soup = bs(url)
    articles = soup.find_all('a', attrs={'class':'description-title-link'})
    time.sleep(sec1)
    for item in articles:
        url_of_article = item.get('href')
        begin = 'https://www.avito.ru'
        clean_url = begin + url_of_article
        soup_page = bs(clean_url)
        text = find_info(soup_page, clean_url)
        write_file(text, url_of_article)
        sec2 = random.uniform(0, 10)
        time.sleep(sec2)